In [40]:
import os
import pandas as pd
from classes.vqa_model_predictor import DefaultVqaModelPredictor
from evaluate.VqaMedEvaluatorBase import VqaMedEvaluatorBase
from common.utils import VerboseTimer
import vqa_logger 
import logging
from pathlib import Path
import datetime
logger = logging.getLogger(__name__)

In [38]:
mp = DefaultVqaModelPredictor.get_contender()

datasets = {'test':mp.df_test, 'validation':mp.df_validation}
df_name_to_predict = 'test'
df_to_predict = datasets['test']

with VerboseTimer(f"Predictions for VQA contender"):
    df_predictions = mp.predict(df_to_predict)


predictions = df_predictions.prediction.values
predictions[:5]

[2019-03-29 16:25:43][common.utils][DEBUG] Starting 'Loading  VQA contender'
[2019-03-29 16:25:43][classes.vqa_model_predictor][DEBUG] Getting model #5 (post_concat_dense_units: 8;
Optimizer: RMSprop
loss: cosine_proximity
activation: sigmoid
prediction vector: answers
lstm_units: 0
batch_size: 64

model_location = 'C:\Users\Public\Documents\Data\2019\models\20190223_2239_45\vqa_model.h5')
[2019-03-29 16:25:43][common.utils][DEBUG] Starting 'Loading Model'
[2019-03-29 16:25:47][common.utils][DEBUG] Loading Model: 0:00:04.107302
[2019-03-29 16:25:47][classes.vqa_model_predictor][DEBUG] Getting model #72 (prediction_vector: words
 question_category: Abnormality
augmentations: 20 
epochs: 10
batch size: 64
based on model 5)
[2019-03-29 16:25:47][common.utils][DEBUG] Starting 'Loading Model'
[2019-03-29 16:25:51][common.utils][DEBUG] Loading Model: 0:00:04.076886
[2019-03-29 16:25:51][root][DEBUG] For Abnormality, got specialized model (DB: 72, Folder: ModelFolder(folder="C:\Users\Public\D

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 124/124 [00:00<00:00, 481.90it/s]


[2019-03-29 16:26:05][common.utils][DEBUG] Starting 'Raw model prediction'
[2019-03-29 16:26:07][common.utils][DEBUG] Raw model prediction: 0:00:02.208924


Prediction: [135, 985, 1121, 113; probabilities: [4.335148e-05, 4.083: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 124/124 [00:00<00:00, 217.02it/s]


[2019-03-29 16:26:09][classes.vqa_model_predictor][INFO] For Category "Modality" using specialized classifier from:
ModelFolder(folder="C:\Users\Public\Documents\Data\2019\models\20190329_0133_06")
[2019-03-29 16:26:09][classes.vqa_model_predictor][DEBUG] Classifying: "Modality"
[2019-03-29 16:26:09][common.functions][DEBUG] Getting image features


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 125/125 [00:00<00:00, 446.03it/s]


[2019-03-29 16:26:16][common.utils][DEBUG] Starting 'Raw model prediction'
[2019-03-29 16:26:18][common.utils][DEBUG] Raw model prediction: 0:00:02.137337


Prediction: [13]; probabilities: [0.6432047]: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 125/125 [00:00<00:00, 231.25it/s]


[2019-03-29 16:26:18][common.utils][DEBUG] Predictions for VQA contender: 0:00:13.480700


array(['cta ct angiography', 'no', 'iv', 'us doppler ultrasound',
       'an angiogram'], dtype=object)

In [9]:
df_output = df_to_predict.copy()
df_output['image_id'] = df_output.path.apply(lambda p: p.rsplit(os.sep)[-1].rsplit('.', 1)[0])
df_output['prediction'] = predictions

columns_to_remove = ['path',  'answer_embedding', 'question_embedding', 'group', 'diagnosis', 'processed_answer']
for col in columns_to_remove:
    del df_output[col]

sort_columns = sorted(df_output.columns, key=lambda c: c not in ['question', 'prediction', 'answer'])
df_output = df_output[sort_columns]    
df_output.sample(10)

,question,answer,prediction,image_name,processed_question,question_category,image_id
15199,what is the primary abnormality in this image?,,fibrous aortic pulmonary dysplasia embolism,synpic54202.jpg,what is the primary abnormality in this image,Abnormality,synpic54202
14967,in what plane is this mri taken?,,sagittal,synpic52142.jpg,in what plane is this mri taken,Plane,synpic52142
15010,what plane was used in this mri?,,axial,synpic43048.jpg,what plane was used in this mri,Plane,synpic43048
14825,what imaging modality is used to acquire this ...,,us ultrasound,synpic46949.jpg,what imaging modality is used to acquire this ...,Modality,synpic46949
15011,which plane is this image in?,,sagittal,synpic27294.jpg,which plane is this image in,Plane,synpic27294
14899,is this a t2 weighted image?,,no,synpic53600.jpg,is this t2 weighted image,Modality,synpic53600
15137,what organ system is pictured here?,,spine and contents,synpic16108.jpg,what organ system is pictured here,Organ,synpic16108
14958,in what plane is this angiogram taken?,,ap,synpic59360.jpg,in what plane is this angiogram taken,Plane,synpic59360
15096,which organ system is shown in the ct scan?,,face sinuses and neck,synpic56103.jpg,which organ system is shown in the ct scan,Organ,synpic56103
15129,what organ system is pictured here?,,gastrointestinal,synpic25139.jpg,what organ system is pictured here,Organ,synpic25139


In [18]:
mp

VqaModelPredictor(model=5, specialized_classifiers={'Abnormality': 72, 'Modality': 69, 'Organ': 70, 'Plane': 71})

In [19]:
len(df_output), len(df_output.image_id.drop_duplicates())

(500, 500)

In [26]:
strs = []
debug_output_rows = df_output.apply(lambda row: row.image_id + '|'+ row.question + '|'+ row.prediction, axis=1 )
output_rows = df_output.apply(lambda row: row.image_id + '|'+ row.prediction, axis=1 )
rows = output_rows.values
res = '\n'.join(rows)

In [34]:
print(res[:200])

synpic54082|cta ct angiography
synpic48556|no
synpic50696|iv
synpic37194|us doppler ultrasound
synpic31308|an angiogram
synpic24739|t1
synpic32365|yes
synpic47936|iv
synpic29792|yes
synpic43561|no
syn


In [50]:
import time
now = time.time()
ts = datetime.datetime.fromtimestamp(now).strftime('%Y%m%d_%H%M_%S')
submission_base_folder = Path('C:\\Users\\Public\\Documents\\Data\\2019\\submissions')
submission_folder = submission_base_folder/ts
submission_folder.mkdir()


txt_path = submission_folder/f'submission_{ts}.txt'
txt_path.write_text(res)


model_description_path = submission_folder/f'model_description.txt'
model_description_path.write_text(repr(mp))




113